In [1]:
import streamlit as st
import time
import re
import random
from pymongo.mongo_client import MongoClient
import pandas
import json
import random
import tqdm as tqdm
import string

In [2]:
# Load the JSON file
with open('data.json', 'r') as file:
    data = json.load(file)
    
uri = st.secrets["mongo"]['uri']
client = MongoClient(uri)
db = client["Replication"]
collexp_code = db.exp_code


In [3]:
nparticipants = 300

BIAS = True

biased = [0.5, 0.16, 0.16, 0.16] #biased
unbiased = [0.25, 0.25, 0.25, 0.25] #biased

cognitive_load_lower = 1
cognitive_load_higher = 4

In [4]:
def generate_prolific_id(length=24):
    characters = string.ascii_uppercase + string.digits
    prolific_id = ''.join(random.choice(characters) for _ in range(length))
    return prolific_id
def generate_study_id(length=24):
    characters = string.ascii_uppercase + string.digits
    study_id = ''.join(random.choice(characters) for _ in range(length))
    return study_id
def generate_session_id(length=24):
    characters = string.ascii_uppercase + string.digits
    session_id = ''.join(random.choice(characters) for _ in range(length))
    return session_id

def generate_dummy_prolific_ids(count=10):
    prolific_ids = [generate_prolific_id() for _ in range(count)]
    return prolific_ids
def generate_dummy_study_ids(count=10):
    study_id = [generate_study_id() for _ in range(count)]
    return study_id
def generate_dummy_session_ids(count=10):
    session_id = [generate_session_id() for _ in range(count)]
    return session_id

def write_prolific_ids_to_file(ids, filename='prolific_ids.txt'):
    with open(filename, 'w') as file:
        for idx, pid in enumerate(ids, start=1):
            file.write(f"{pid}\n")
            
def write_study_ids_to_file(ids, filename='study_id.txt'):
    with open(filename, 'w') as file:
        for idx, pid in enumerate(ids, start=1):
            file.write(f"{pid}\n")
            
def write_session_ids_to_file(ids, filename='session_id.txt'):
    with open(filename, 'w') as file:
        for idx, pid in enumerate(ids, start=1):
            file.write(f"{pid}\n")

# write_prolific_ids_to_file(generate_dummy_prolific_ids(nparticipants))
# write_study_ids_to_file(generate_dummy_study_ids(nparticipants))
# write_session_ids_to_file(generate_dummy_session_ids(nparticipants))


In [5]:
with open('prolific_ids.txt', 'r') as file:
    # Read the contents of the file
    prolific_ids = file.read().split()
    
with open('study_id.txt', 'r') as file:
    # Read the contents of the file
    study_ids = file.read().split()
    
with open('session_id.txt', 'r') as file:
    # Read the contents of the file
    session_ids = file.read().split()

In [6]:
def updateRecord(key, value, newValue):
    criteria = {key: value }
    update_operation = {"$set": {key: newValue}}
    return collexp_code.update_one(criteria, update_operation, upsert=False)

def getExpCode(value_to_find = 'EMPTY'):

    # The specific value you're looking for

    def contains_value(doc, value):
        # Recursively check if any field in the document contains the value
        if isinstance(doc, dict):
            for key, val in doc.items():
                if contains_value(val, value):
                    return True
        elif isinstance(doc, list):
            for item in doc:
                if contains_value(item, value):
                    return True
        else:
            return doc == value
        return False

    # Iterate over all documents in the collection
    for doc in collexp_code.find():
        if contains_value(doc, value_to_find):
            return list(doc)[1]
    else:
        return -1 



In [7]:
def getAlternative(weights = [0.25, 0.25, 0.25, 0.25], sqcondition = "NEUT"):
    # Define the list of alternatives
    alternatives = ['a', 'b', 'c', 'd']

    # Perform biased random selection
    selected_alternative = random.choices(alternatives, weights=weights, k=1)[0]

    
    # if sqcondition == "SQ2":
    #     if selected_alternative == 'a':
    #         selected_alternative = 'b'
    #     elif selected_alternative == 'b':
    #         selected_alternative = 'a'
    # if sqcondition == "SQ3":
    #     if selected_alternative == 'a':
    #         selected_alternative = 'c'
    #     elif selected_alternative == 'c':
    #         selected_alternative = 'a'
    # if sqcondition == "SQ4":
    #     if selected_alternative == 'a':
    #         selected_alternative = 'd'
    #     elif selected_alternative == 'd':
    #         selected_alternative = 'a'

    return selected_alternative


def chatdata(expcode, data):
    expcode_1 = str(expcode).split('_')[1].split('&')[0].replace("$", "_")
    expcode_2 = str(expcode).split('_')[1].split('&')[1].replace("$", "_")
    expcode_3 = str(expcode).split('_')[1].split('&')[2].replace("$", "_")


    f = open(f"../utterances/{expcode_1}.txt", "r")
    chat1_utterances = f.read().split("\n")
    
    f = open(f"../utterances/{expcode_2}.txt", "r")
    chat2_utterances = f.read().split("\n")
    
    f = open(f"../utterances/{expcode_3}.txt", "r")
    chat3_utterances = f.read().split("\n")
    
    counter = 0 
    for i, obj in enumerate(data['chat1']): 
        if obj['role'] == 'assistant' and counter < len(chat1_utterances):
            data['chat1'][i]['content'] = chat1_utterances[counter]
            counter+=1
    counter = 0 
    for i, obj in enumerate(data['chat2']): 
        if obj['role'] == 'assistant' and counter < len(chat2_utterances):
            data['chat2'][i]['content'] = chat2_utterances[counter]
            counter+=1
    counter = 0 
    for i, obj in enumerate(data['chat3']): 
        if obj['role'] == 'assistant' and counter < len(chat3_utterances):
            data['chat3'][i]['content'] = chat3_utterances[counter]
            counter+=1
            
    #if neut condition generate at random
    #if sqb condition generate biased
            
    for i, obj in enumerate(data['chat1']): 
        if obj['role'] == 'user' and i == len(data['chat1'])-2:
            if expcode_1.find("NEUT") > 0:
                data['chat1'][i]['content'] = getAlternative(unbiased) #random
            else:
                if BIAS:
                    data['chat1'][i]['content'] =  getAlternative(biased, expcode_1.split('_')[1]) #bias
                else:
                    data['chat1'][i]['content'] = getAlternative(unbiased) #random
    
    for i, obj in enumerate(data['chat2']): 
        if obj['role'] == 'user' and i == len(data['chat2'])-2:
            if expcode_2.find("NEUT") > 0:
                data['chat2'][i]['content'] = getAlternative(unbiased) #random
            else:
                if BIAS:
                    data['chat2'][i]['content'] =  getAlternative(biased, expcode_2.split('_')[1]) #bias
                else:
                    data['chat2'][i]['content'] = getAlternative(unbiased) #random           
    
    for i, obj in enumerate(data['chat3']): 
        if obj['role'] == 'user' and i == len(data['chat3'])-2:
            if expcode_3.find("NEUT") > 0:
                data['chat3'][i]['content'] = getAlternative(unbiased) #random
            else:
                if BIAS:
                    data['chat3'][i]['content'] =  getAlternative(biased, expcode_3.split('_')[1]) #bias
                else:
                    data['chat3'][i]['content'] = getAlternative(unbiased) #random
                    
    return data
    

def getData(prolific_pid, study_id, session_id, expcode, data):
    data = chatdata(expcode, data)
    data["prolific_pid"] = prolific_pid
    data["study_id"] = study_id
    data["session_id"] = session_id
    data["consent"] = True
    data["expcode"] = expcode
    # data["chat1"] = chatdata()
    # data["chat2"] = chatdata()
    # data["chat3"] = chatdata()
    data["ntlx1"] = {"Mental Demand" : random.randint(cognitive_load_lower, cognitive_load_higher), "Physical Demand" : random.randint(cognitive_load_lower, cognitive_load_higher),
                     "Temporal Demand" : random.randint(cognitive_load_lower, cognitive_load_higher), "Performance" : random.randint(cognitive_load_lower, cognitive_load_higher),
                     "Effort" : random.randint(cognitive_load_lower, cognitive_load_higher), "Frustration" : random.randint(cognitive_load_lower, cognitive_load_higher)}
    data["ntlx2"] = {"Mental Demand" : random.randint(cognitive_load_lower, cognitive_load_higher), "Physical Demand" : random.randint(cognitive_load_lower, cognitive_load_higher),
                     "Temporal Demand" : random.randint(cognitive_load_lower, cognitive_load_higher), "Performance" : random.randint(cognitive_load_lower, cognitive_load_higher),
                     "Effort" : random.randint(cognitive_load_lower, cognitive_load_higher), "Frustration" : random.randint(cognitive_load_lower, cognitive_load_higher)}
    data["ntlx3"] = {"Mental Demand" : random.randint(cognitive_load_lower, cognitive_load_higher), "Physical Demand" : random.randint(cognitive_load_lower, cognitive_load_higher),
                     "Temporal Demand" : random.randint(cognitive_load_lower, cognitive_load_higher), "Performance" : random.randint(cognitive_load_lower, cognitive_load_higher),
                     "Effort" : random.randint(cognitive_load_lower, cognitive_load_higher), "Frustration" : random.randint(cognitive_load_lower, cognitive_load_higher)}
    data["ac1"] = True
    data["ac2"] = True
    data["ac3"] = True
    data["serious"] = "7"
    return json.dumps(data, indent=4)

In [8]:
for i in tqdm.tqdm(range(0, nparticipants)):
    expcode = getExpCode()
    tdata = getData(prolific_ids[i], study_ids[i], session_ids[i], expcode, data)
    updateRecord(expcode, "EMPTY" , tdata)

100%|██████████| 300/300 [17:01<00:00,  3.40s/it]
